# CatBoost

In [ ]:
import pandas as pd 
from catboost import CatBoostClassifier
from mypipes_linear import *
from sklearn.model_selection import train_test_split
import warnings 
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
file=r'/Users/lalitsachan/Dropbox/0.0 Data/rg_train.csv'
data=pd.read_csv(file)

In [ ]:
data.info()

In [ ]:
num_vars=list(data.select_dtypes(exclude=['object']).columns)

num_vars=[_ for _ in num_vars if _ not in ['REF_NO','Revenue.Grid']]

In [ ]:
cat_vars=list(data.select_dtypes(include=['object']).columns)

In [ ]:
cat_vars=[_ for _ in cat_vars if _ not in 
          ['children','age_band', 'post_code','post_area','family_income']]

In [ ]:
data_train,data_val=train_test_split(data,test_size=0.2,random_state=42)

In [ ]:
p1=pdPipeline([
    ('var_select',VarSelector(num_vars)),
    ('missing_trt',DataFrameImputer())
])

p2=pdPipeline([
    ('var_select',VarSelector(cat_vars)),
    ('missing_trt',DataFrameImputer())
])

p3=pdPipeline([
    ('var_select',VarSelector(['age_band'])),
    ('custom_fico',custom_age_band()),
    ('missing_trt',DataFrameImputer())
])

p4=pdPipeline([
    ('var_select',VarSelector(['family_income'])),
    ('custom_fico',custom_family_income()),
    ('missing_trt',DataFrameImputer())
])

p5=pdPipeline([
    ('var_select',VarSelector(['children'])),
    ('string_clean1',string_clean(replace_it='Zero',replace_with='0')),
    ('string_clean2',string_clean(replace_it='4+',replace_with='4')),
    ('convert_to_numeric',convert_to_numeric()),
    ('missing_trt',DataFrameImputer())
])

data_pipe=FeatureUnion([
    ('num',p1),
    ('cat_vars',p2),
    ('age_band',p3),
    ('family_income',p4),
    ('children',p5)
])

In [ ]:
data_pipe.fit(data_train)

In [ ]:
x_train=pd.DataFrame(data=data_pipe.transform(data_train),
                     columns=data_pipe.get_feature_names())
x_val=pd.DataFrame(data=data_pipe.transform(data_val),
                     columns=data_pipe.get_feature_names())

y_train=(data_train['Revenue.Grid']==1).astype(int)
y_val=(data_val['Revenue.Grid']==1).astype(int)

In [ ]:
x_train.head()

In [ ]:
cat_vars_cb=['cat_vars__'+_ for _ in cat_vars]

In [ ]:
clf = CatBoostClassifier(
    iterations=200,
    random_seed=42,
    learning_rate=0.5
)

clf.fit(
    x_train, y_train,
    cat_features=cat_vars_cb,
    eval_set=(x_val, y_val),
    plot=True
)

In [ ]:
clf.shrink(80)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y_val,clf.predict_proba(x_val)[:,1])